In [2]:
import pandas as pd
import numpy as np
import pickle
from ase import Atoms
from dscribe.descriptors import ACSF

In [ ]:
Atoms(symbols,positions)

In [ ]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')
structures = pd.read_csv('../Data/structures.csv')
train_bonds = pd.read_csv('../Data/train_bonds.csv')
test_bonds = pd.read_csv('../Data/test_bonds.csv')

In [ ]:
test_bonds = test_bonds.drop('Unnamed: 0',1)
train_bonds = train_bonds.drop('Unnamed: 0',1)

In [ ]:
train.head(10)

In [ ]:
structures.head()

In [ ]:
train_bonds.head()

In [ ]:
test_bonds.head()

### node information

In [ ]:
structures[['C', 'F', 'H', 'N', 'O']] = pd.get_dummies(structures.atom)
structures = structures.sort_values(by=['molecule_name', 'atom_index'])

In [ ]:
structures_gb = structures.groupby(['molecule_name'])

In [ ]:
structures_dict = {}
for k,v in structures_gb:
    atom_dict = {'positions':v[['x','y','z']].values.tolist(),
                 'symbols':[i[0] for i in v[['atom']].values.tolist()]
                 }
    atom = Atoms(**atom_dict)
    dist = atom.get_all_distances()
    d = dist.shape[0]
    dist = dist[np.tril_indices(d,-1)]
    r_info = np.array([dist.max(),dist.min(),dist.mean(),dist.std()])
    
    acsf = ACSF(species=['C', 'F', 'H', 'N', 'O'],
                rcut=r_info[0]+0.5,
                g2_params=[[1, 1], [1, 2], [1, 3]],
                g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])
    acsf_np = acsf.create(atom)
    structures_dict[k] = np.concatenate([acsf_np,v[['C', 'F', 'H', 'N', 'O']].values,\
                                         np.tile(r_info,(d,1))],1).astype(np.float32)

### bond information

In [ ]:
assert set(train_bonds.bond_type) == set(test_bonds.bond_type)

In [ ]:
bonds = train_bonds.append(test_bonds, ignore_index=True)

In [ ]:
del train_bonds,test_bonds

In [ ]:
bonds[['1.0CC',
 '1.0CF',
 '1.0CH',
 '1.0CN',
 '1.0CO',
 '1.0HN',
 '1.0HO',
 '1.0NN',
 '1.0NO',
 '1.5CO',
 '2.0CC',
 '2.0CN',
 '2.0CO',
 '2.0NN',
 '2.0NO',
 '3.0CC',
 '3.0CN']] = pd.get_dummies(bonds.bond_type)

In [ ]:
bonds.head()

In [ ]:
bonds_gb = bonds.groupby(['molecule_name'])

In [ ]:
bonds_edge_index = {}
bonds_edge_attr = {}
for k,v in bonds_gb:
    bonds_edge_index[k] = np.concatenate([v[['atom_index_0','atom_index_1']].values,\
                                          v[['atom_index_1','atom_index_0']].values]).T
    bonds_edge_attr[k] = np.tile(v[['L2dist', 'error',\
                                   '1.0CC', '1.0CF', '1.0CH', '1.0CN', '1.0CO','1.0HN',\
                                   '1.0HO', '1.0NN', '1.0NO', '1.5CO', '2.0CC', '2.0CN',\
                                   '2.0CO', '2.0NN', '2.0NO', '3.0CC', '3.0CN']].values.astype(np.float32),(2,1))

### coupling information

In [ ]:
assert set(train.type) == set(test.type)

In [ ]:
test['scalar_coupling_constant'] = np.nan

In [ ]:
coupling = train.append(test, ignore_index=True)

In [ ]:
coupling.head()

In [ ]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [ ]:
coupling = map_atom_info(coupling, 0)
coupling = map_atom_info(coupling, 1)

In [ ]:
coupling = coupling.drop(['atom_0', 'C_x', 'F_x',
                          'H_x', 'N_x', 'O_x', 'atom_1','C_y', 'F_y', 'H_y',
                          'N_y', 'O_y'], axis=1)

In [ ]:
train_p_0 = coupling[['x_0', 'y_0', 'z_0']].values
train_p_1 = coupling[['x_1', 'y_1', 'z_1']].values
coupling['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)

In [ ]:
coupling[['1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN']]=pd.get_dummies(coupling.type)

In [ ]:
coupling = coupling.groupby(['molecule_name'])

In [ ]:
coupling_edge_index = {}
coupling_edge_attr = {}
coupling_edge_dist = {}
coupling_y = {}
for k,v in coupling:
    coupling_edge_index[k] = v[['atom_index_0','atom_index_1']].values.T
    coupling_edge_attr[k] = v[['1JHC','1JHN','2JHC','2JHH','2JHN','3JHC','3JHH','3JHN']].values.astype(np.float32)
    coupling_edge_dist[k] = v[['dist']].values.astype(np.float32)
    if not np.any(np.isnan(v.scalar_coupling_constant.values)):
        coupling_y[k] = v.scalar_coupling_constant.values.astype(np.float32)

In [ ]:
train_mol = np.unique(train.molecule_name)
test_mol = np.unique(test.molecule_name)

In [ ]:
train_mol = np.random.permutation(train_mol)

In [ ]:
train_mol2 = train_mol[:70000]
val_mol = train_mol[70000:]
train_mol = train_mol2

In [ ]:
train_data = [{'x':structures_dict[m],'edge_index':bonds_edge_index[m],\
               'edge_attr':bonds_edge_attr[m],'y':coupling_y[m],\
               'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
               'edge_attr4':coupling_edge_dist[m]}
              for m in train_mol]

In [ ]:
val_data = [{'x':structures_dict[m],'edge_index':bonds_edge_index[m],\
               'edge_attr':bonds_edge_attr[m],'y':coupling_y[m],\
               'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
               'edge_attr4':coupling_edge_dist[m]}
              for m in val_mol]

In [ ]:
test_data = [{'x':structures_dict[m],'edge_index':bonds_edge_index[m],\
               'edge_attr':bonds_edge_attr[m],\
               'edge_index3':coupling_edge_index[m],'edge_attr3':coupling_edge_attr[m],\
               'edge_attr4':coupling_edge_dist[m]}
              for m in test_mol]

In [ ]:
with open('../Data/train_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/val_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/test_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [4]:
# convert numpy array to torch array
import torch
train_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in train_data]
val_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in val_data]
test_data = [{k:torch.tensor(i[k]) for k in i.keys()} for i in test_data]

In [5]:
with open('../Data/train_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/val_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(val_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/test_data_ACSF.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)